In [1]:
import pandas as pd 
import numpy as np
import sys
sys.path.append('..')
import Classification.resources.preprocessing as pre
import Classification.resources.setup as setup

In [2]:
df = pd.read_csv('../Classification/files/dataset.csv', index_col='id')
df['class'].value_counts()
df.rename(columns={'class': 'label'}, inplace=True)

In [3]:
stopwords = pd.read_csv('../Classification/files/polish_stopwords.txt', header=None)
config = setup.config('', '', '', stopwords)

In [4]:
df = pre.meta_features(df, stopwords)
train_x, train_y, test_x, test_y, scaler, vectorizer, selector = pre.vectorize_dataset(config, df)

In [17]:
vocab = vectorizer.vocabulary_
selected = selector.get_support()
keys = vocab.keys()
keys = np.array(list(keys))
ngrams = keys[selected]
np.savetxt("ngrams_7500.txt", ngrams, fmt='%s', delimiter='\n')

(292, 7506)

In [9]:
import tensorflow as tf

model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(128, input_dim=7506, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01)),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(64, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01)),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(32, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01)),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(1, activation='sigmoid' )
])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

2024-03-16 08:51:05.577734: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-16 08:51:05.889181: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-16 08:51:05.889243: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-16 08:51:05.909570: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-16 08:51:05.945907: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-16 08:51:05.946870: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [10]:
model.fit(train_x, train_y, epochs=5, verbose=1, validation_data=(test_x, test_y))

Epoch 1/5


2024-03-16 08:51:09.918341: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 78813000 exceeds 10% of free system memory.


83/83 [==============================] - 2s 15ms/step - loss: 1.6334 - accuracy: 0.6697 - val_loss: 0.9692 - val_accuracy: 0.7089
Epoch 2/5
83/83 [==============================] - 1s 13ms/step - loss: 0.7747 - accuracy: 0.7989 - val_loss: 0.5997 - val_accuracy: 0.8904
Epoch 3/5
83/83 [==============================] - 1s 13ms/step - loss: 0.5458 - accuracy: 0.9131 - val_loss: 0.5057 - val_accuracy: 0.9349
Epoch 4/5
83/83 [==============================] - 1s 12ms/step - loss: 0.4814 - accuracy: 0.9261 - val_loss: 0.4520 - val_accuracy: 0.9315
Epoch 5/5
83/83 [==============================] - 1s 13ms/step - loss: 0.4428 - accuracy: 0.9352 - val_loss: 0.4739 - val_accuracy: 0.9110


In [11]:
layers = model.layers
new_model = tf.keras.models.Sequential(layers[:-1])

In [12]:
new_model.input_shape

(None, 7506)

In [13]:
x_data = np.concatenate((train_x, test_x), axis=0)
y_data = np.concatenate((train_y, test_y), axis=0)
x_values = new_model(x_data)

2024-03-16 08:51:17.047580: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 175160016 exceeds 10% of free system memory.
2024-03-16 08:51:17.114380: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 87580008 exceeds 10% of free system memory.


In [14]:
from sklearn.manifold import TSNE

In [15]:
tsne = TSNE(n_components=2, perplexity=50, n_iter=10000, verbose=1, random_state=123)
v = tsne.fit_transform(x_values) 

[t-SNE] Computing 151 nearest neighbors...
[t-SNE] Indexed 2917 samples in 0.000s...
[t-SNE] Computed neighbors for 2917 samples in 0.121s...
[t-SNE] Computed conditional probabilities for sample 1000 / 2917
[t-SNE] Computed conditional probabilities for sample 2000 / 2917
[t-SNE] Computed conditional probabilities for sample 2917 / 2917
[t-SNE] Mean sigma: 0.016734
[t-SNE] KL divergence after 250 iterations with early exaggeration: 48.408718
[t-SNE] KL divergence after 9950 iterations: 0.196370


In [18]:
df_tsne = pd.DataFrame()
df_tsne["class"] = y_data
df_tsne["x"] = v[:,0]
df_tsne["y"] = v[:,1]
df_tsne["text"] = df["text"]
df_tsne["id"] = df.index
df_tsne.to_csv("../Classification/plots/tsne.csv")